In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


After discovering the issue with the Police API, we moved on to working on a plan to reverse geocode the longitude and latitude coordinates, extracting 'suburb' or 'district' from the 'results'.

In [ ]:
# a test, using one set of coordinates from the Crime dataset, to confirm that we could connect to the API, and return data on the location from  coordinates

import requests
import json
from pprint import pprint as pp

appid = '0105cebf224c439f84677f58d9259503'
lat = '51.542511757487524'
lon = '-0.05702092689608372'
url = f'https://api.geoapify.com/v1/geocode/reverse?lat={lat}&lon={lon}&format=json&apiKey={appid}'
response = requests.get(url)

pp(response.json())

{'results': [{'address_line1': '37 Ellingfort Road',
              'address_line2': 'London, E8 3PA, United Kingdom',
              'bbox': {'lat1': 51.542419853846,
                       'lat2': 51.542519853846,
                       'lon1': -0.057047653846154,
                       'lon2': -0.056947653846154},
              'city': 'London',
              'country': 'United Kingdom',
              'country_code': 'gb',
              'county': 'Greater London',
              'datasource': {'attribution': '© OpenStreetMap contributors',
                             'license': 'Open Database License',
                             'sourcename': 'openstreetmap',
                             'url': 'https://www.openstreetmap.org/copyright'},
              'distance': 4.929618659949136,
              'district': 'London Borough of Hackney',
              'formatted': '37 Ellingfort Road, London, E8 3PA, United Kingdom',
              'housenumber': '37',
              'lat': 51.542469853

In [ ]:
# a function that would take the longitude and latitude from the crime dataset, and return the 'suburb'

def nbh_from_lon_lat(lon, lat):
    try:
        url = f'https://api.geoapify.com/v1/geocode/reverse?lat={lat}&lon={lon}&format=json&apiKey={appid}'
        response = requests.get(url)
        answer = response.json()
        suburb = answer['results'][0]['suburb']
        return suburb
    except:
        return 'isna'

nbh_from_lon_lat(lon, lat)

'Shacklewell'

In [ ]:
# an attempt to create a new column with the 'suburb' of each crime. This runs, but does not finish, as it takes too long. 

nbh_from_lon_lat(lon, lat)

police['neighbourhood_name']=police.apply(lambda police:nbh_from_lon_lat(police['Longitude'], police['Latitude']),axis=1)

At this point, multiple members of the group worked on this code, trying to use the longitude and latitude to reverse geocode. 

In [ ]:
import pandas as pd

police_street_dataset = pd.read_csv('/content/gdrive/MyDrive/Group_7_Files/archived_files/lsoa_names_cleaned.csv')

long_lat_list = police_street_dataset[['Longitude', 'Latitude']].values.tolist()

long_lat_list

[[0.142112, 51.589389],
 [0.142112, 51.589389],
 [0.140194, 51.582356],
 [0.13883, 51.583433],
 [0.13883, 51.583433],
 [0.140576, 51.583419],
 [0.136416, 51.584898],
 [0.136416, 51.584898],
 [0.140576, 51.583419],
 [0.138781, 51.589468],
 [0.145722, 51.594296],
 [0.136416, 51.584898],
 [0.141144, 51.5909],
 [0.135579, 51.584913],
 [0.136513, 51.588214],
 [0.138781, 51.589468],
 [0.140194, 51.582356],
 [0.145722, 51.594296],
 [0.13883, 51.583433],
 [0.148434, 51.595164],
 [0.145722, 51.594296],
 [0.13883, 51.583433],
 [0.136416, 51.584898],
 [0.13883, 51.583433],
 [0.13883, 51.583433],
 [0.13883, 51.583433],
 [0.12781, 51.584227],
 [0.127656, 51.585579],
 [0.130601, 51.586802],
 [0.128436, 51.583415],
 [0.126967, 51.586571],
 [0.127656, 51.585579],
 [0.130847, 51.586204],
 [0.128436, 51.583415],
 [0.128436, 51.583415],
 [0.129102, 51.584689],
 [0.130847, 51.586204],
 [0.129102, 51.584689],
 [0.130847, 51.586204],
 [0.130847, 51.586204],
 [0.138643, 51.578526],
 [0.130654, 51.584229],
 [

In [ ]:
# an evolution of the previous codes, converting the longitude and latitude to 'district' (which we found to be more in line with the Airbnb boroughs than suburb, 
# which were too small) in batches

import requests

def nbh_from_lon_lat(coords):
    api_key = "bb8c2d188f8641698c83bf0dbe6d4999" 
    url = f'https://api.geoapify.com/v1/geocode/reverse?format=json&apiKey={api_key}'

    districts = []

    batch_size = 10
    for i in range(0, len(coords), batch_size):
        batch_coords = coords[i:i+batch_size]

        batch_districts = []
        for lon, lat in batch_coords:
            params = {'lon': lon, 'lat': lat}
            response = requests.get(url, params=params)
            answer = response.json()
            if 'district' in answer['results'][0]:
                district = answer['results'][0]['district']
            else:
                district = None
            batch_districts.append(district)

        districts.extend(batch_districts)

    return districts

districts = nbh_from_lon_lat(long_lat_list)

police_street_dataset['district'] = districts

print(police_street_dataset)